In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import glob
%matplotlib inline

Using TensorFlow backend.


In [2]:
import os
os.getcwd()

'C:\\Users\\JanSulaiman\\Downloads\\MLND_Capstone_Project_2019\\notebooks'

In [3]:
from keras.preprocessing.image import ImageDataGenerator

#Image Data Generator w/ no augmentation
#Scaling for pixels
piece_train_datagen = ImageDataGenerator(
    rescale = 1./255)
piece_test_datagen = ImageDataGenerator(
    rescale = 1./255)
piece_valid_datagen = ImageDataGenerator(
    rescale = 1./255)


#Flow data from directory

piece_train_iter = piece_train_datagen.flow_from_directory(
    directory = '../data/piece_data/train',
    target_size = (135,135),
    color_mode = 'grayscale',
    class_mode = 'categorical',
    seed=42
)

piece_test_iter = piece_test_datagen.flow_from_directory(
    directory = '../data/piece_data/test',
    target_size = (135,135),
    color_mode = 'grayscale',
    class_mode = 'categorical',
    shuffle=False,
    seed=42
)

piece_valid_iter = piece_valid_datagen.flow_from_directory(
    directory = '../data/piece_data/valid',
    target_size = (135,135),
    color_mode = 'grayscale',
    class_mode = 'categorical',
    seed=42
)

Found 1227 images belonging to 7 classes.
Found 335 images belonging to 7 classes.
Found 331 images belonging to 7 classes.


In [4]:
#Define NN architecture

from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

piece_model = Sequential()
piece_model.add(Conv2D(filters=16, kernel_size=5, padding='same', activation='relu', 
                        input_shape=(135, 135, 1)))
piece_model.add(MaxPooling2D(pool_size=2))
piece_model.add(BatchNormalization())
piece_model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
piece_model.add(MaxPooling2D(pool_size=2))
piece_model.add(BatchNormalization())
piece_model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
piece_model.add(MaxPooling2D(pool_size=2))
piece_model.add(BatchNormalization())
piece_model.add(Conv2D(filters=128, kernel_size=2, padding='same', activation='relu'))
piece_model.add(Conv2D(filters=128, kernel_size=2, padding='same', activation='relu'))
piece_model.add(MaxPooling2D(pool_size=2))
piece_model.add(BatchNormalization())
piece_model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
piece_model.add(MaxPooling2D(pool_size=2))
piece_model.add(BatchNormalization())
piece_model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
piece_model.add(MaxPooling2D(pool_size=2))
piece_model.add(BatchNormalization())
piece_model.add(Dropout(0.3))
piece_model.add(GlobalAveragePooling2D())
piece_model.add(Dense(2048, activation='relu'))
piece_model.add(Dense(2048, activation='relu'))
piece_model.add(Dropout(0.4))
piece_model.add(Dense(7, activation='softmax'))


piece_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 135, 135, 16)      416       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 67, 67, 16)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 67, 67, 16)        64        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 67, 67, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 33, 33, 32)        0         
_________________________________________________________________
batch_normalization_2

In [5]:
# compile the model
from keras.optimizers import RMSprop
piece_model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.00001), 
                  metrics=['accuracy'])

STEP_SIZE_TRAIN = piece_train_iter.n/piece_train_iter.batch_size
STEP_SIZE_VALID = piece_valid_iter.n/piece_valid_iter.batch_size



In [6]:
from keras.callbacks import ModelCheckpoint   

# train the model
checkpointer = ModelCheckpoint(filepath='piece_model_1.weights.best.hdf5', verbose=1, 
                               save_best_only=True)
piece_hist = piece_model.fit_generator(generator=piece_train_iter, 
                          steps_per_epoch=STEP_SIZE_TRAIN, 
                          validation_data=piece_valid_iter, 
                          validation_steps=STEP_SIZE_VALID,
                          epochs=1500, 
                          callbacks=[checkpointer], 
                          verbose=2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1500
 - 34s - loss: 1.8781 - acc: 0.2727 - val_loss: 1.8458 - val_acc: 0.2598

Epoch 00001: val_loss improved from inf to 1.84577, saving model to piece_model_1.weights.best.hdf5
Epoch 2/1500
 - 29s - loss: 1.7968 - acc: 0.2849 - val_loss: 1.7978 - val_acc: 0.2598

Epoch 00002: val_loss improved from 1.84577 to 1.79776, saving model to piece_model_1.weights.best.hdf5
Epoch 3/1500
 - 26s - loss: 1.7435 - acc: 0.3085 - val_loss: 1.7608 - val_acc: 0.2689

Epoch 00003: val_loss improved from 1.79776 to 1.76076, saving model to piece_model_1.weights.best.hdf5
Epoch 4/1500
 - 26s - loss: 1.7047 - acc: 0.3245 - val_loss: 1.7267 - val_acc: 0.2779

Epoch 00004: val_loss improved from 1.76076 to 1.72666, saving model to piece_model_1.weights.best.hdf5
Epoch 5/1500
 - 29s - loss: 1.6575 - acc: 0.3539 - val_loss: 1.6949 - val_acc: 0.2900

Epoch 00005: val_loss improved from 1.72666 to 1.69490, saving model to piece_model_1.weights.best.hdf5
E

 - 30s - loss: 0.9293 - acc: 0.6754 - val_loss: 1.3787 - val_acc: 0.4471

Epoch 00045: val_loss did not improve from 1.37637
Epoch 46/1500
 - 30s - loss: 0.9362 - acc: 0.6740 - val_loss: 1.3744 - val_acc: 0.4532

Epoch 00046: val_loss improved from 1.37637 to 1.37436, saving model to piece_model_1.weights.best.hdf5
Epoch 47/1500
 - 29s - loss: 0.8723 - acc: 0.7030 - val_loss: 1.3755 - val_acc: 0.4562

Epoch 00047: val_loss did not improve from 1.37436
Epoch 48/1500
 - 30s - loss: 0.8582 - acc: 0.7033 - val_loss: 1.3770 - val_acc: 0.4441

Epoch 00048: val_loss did not improve from 1.37436
Epoch 49/1500
 - 31s - loss: 0.8485 - acc: 0.7168 - val_loss: 1.3608 - val_acc: 0.4683

Epoch 00049: val_loss improved from 1.37436 to 1.36076, saving model to piece_model_1.weights.best.hdf5
Epoch 50/1500
 - 31s - loss: 0.8560 - acc: 0.7008 - val_loss: 1.3520 - val_acc: 0.4924

Epoch 00050: val_loss improved from 1.36076 to 1.35199, saving model to piece_model_1.weights.best.hdf5
Epoch 51/1500
 - 30s 

KeyboardInterrupt: 

In [ ]:
#Reset test iterator
STEP_SIZE_TEST = piece_test_iter.n/piece_test_iter.batch_size
piece_test_iter.reset()
# load the weights that yielded the best validation accuracy
piece_model.load_weights('piece_model.weights.best.hdf5')
# evaluate and print test accuracy
score = piece_model.evaluate_generator(generator=piece_test_iter,steps=STEP_SIZE_TEST)
print('\n', 'Test accuracy:', score[1])

In [ ]:
piece_test_iter.reset()
piece_pred = piece_model.predict_generator(piece_test_iter,steps=STEP_SIZE_TEST,verbose=1)

In [ ]:
predicted_class_indices=np.argmax(piece_pred,axis=1)
print(predicted_class_indices)

In [ ]:
labels = (piece_test_iter.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
truth =  [labels[k] for k in piece_test_iter.classes]

In [ ]:
[labels[k] for k in piece_test_iter.classes]


In [ ]:
piece_test_iter.filenames

In [ ]:
filenames=piece_test_iter.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Truth": truth,
                      "Predictions":predictions})

In [ ]:
results

In [ ]:
from sklearn.metrics import classification_report
class_report = classification_report(piece_test_iter.classes,predicted_class_indices)
print(class_report)

In [ ]:
from sklearn.metrics import confusion_matrix
cmatrix = confusion_matrix(piece_test_iter.classes,predicted_class_indices)
print(cmatrix)
print(labels)

In [ ]:
import seaborn as sns
ax = sns.heatmap(cmatrix, annot=True, xticklabels=['bishop','king','pawn','knight','queen','rook','square'],yticklabels=['bishop','king','pawn','knight','queen','rook','square'])

In [ ]:
plt.plot(piece_hist.history['acc'])
plt.plot(piece_hist.history['val_acc'])
plt.title('Piece Model accuracy no augmentation')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(piece_hist.history['loss'])
plt.plot(piece_hist.history['val_loss'])
plt.title('Piece Model Log Loss no augmentation')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
from sklearn.metrics import log_loss
log_loss_result = log_loss(piece_test_iter.classes,piece_pred)
print(log_loss_result)